In [ ]:
!pip install --upgrade PyGithub tqdm arrow python-dotenv

In [1]:
from github import Github, RateLimitExceededException
import pandas as pd
import json
from tqdm.notebook import trange, tqdm
import arrow
from time import sleep
import pickle
from dotenv import load_dotenv
import os

load_dotenv()
GITHUB_KEY = os.getenv("GITHUB_KEY")

In [2]:
g = Github(GITHUB_KEY)

In [3]:
#arrow.get(g.rate_limiting_resettime)

In [4]:
g.rate_limiting

(5000, 5000)

In [5]:
repo = g.get_repo("kubernetes/kubernetes")

In [ ]:
github_issues = repo.get_issues(state='all')
total_issues = github_issues.totalCount
issues = {}

# Downloading Issues

In [ ]:
for i_issue in trange(total_issues, desc='GitHub'):
    n_issue = github_issues[i_issue].number
    if n_issue not in issues:
        issues[n_issue] = github_issues[i_issue].raw_data
        sleep(1)

In [ ]:
with open('issues.pickle', 'wb') as handle:
    pickle.dump(issues, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

with open("issues_list.json", "w") as handle:
    json.dump(list(issues.values()), handle)

In [ ]:

with open("issues.json", "w") as handle:
    json.dump(issues, handle)

# Downloading Pull Requets

In [ ]:
github_pulls = repo.get_pulls(state='all',sort='updated',direction="desc")
total_pulls = github_pulls.totalCount
pulls = {}
print(total_pulls)

In [ ]:
for i_pull in trange(total_pulls, desc='GitHub Pull Requests'):
    n_pull = github_pulls[i_pull].number
    if n_pull not in pulls:
        pulls[n_pull] = github_pulls[i_pull].raw_data
        sleep(1)

In [ ]:
with open('pull_requests.pickle', 'wb') as handle:
    pickle.dump(pulls, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Downloading Commits

In [54]:
github_commits = repo.get_commits()
total_commits = github_commits.totalCount
print(total_commits)

85866


## Load commits

In [41]:
try:
    with open('../resources/commits.pickle', 'rb') as handle:
        commits = pickle.load(handle)
        print(f"Loaded {len(commits.values())} commits from pickle")
except:
    print("Fail in load commits pickle, commit will be empty")
    commits = {}

## Getting from Github commits and adding to commits dict

In [66]:
#commit_test = github_commits.get_page(2500)[0]
#commit_pages = {}
for i in trange(int(total_commits/30)+1, desc='GitHub Page Commits'):
    if i not in commit_pages:
        #commit_pages[i] = github_commits.get_page(i)
        for c_commit in github_commits.get_page(i):
            sha_commit = c_commit.sha
            if sha_commit not in commits:
                commits[sha_commit] = c_commit.raw_data
                sleep(1)
            else:
                break

        

In [68]:
print(f"Now there are {len(commits.values())} commits on pickle")

Now there are 85813 commits on pickle


## Saving pickle for post-processing

In [69]:
with open('../resources/commits.pickle', 'wb') as handle:
    pickle.dump(commits, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Releases

In [71]:
github_releases = repo.get_releases()
total_releases = github_releases.totalCount
print(total_releases)

353


In [ ]:
github_releases.get_page(0)


In [ ]:
print("hola")

## Saving pickle for post-processing

In [42]:
for c_commit_page in tqdm(commit_pages.values(), desc='GitHub Page Commits'):
    for c_commit in c_commit_page:
        sha_commit = c_commit.sha
        if sha_commit not in commits:
            commits[sha_commit] = c_commit.raw_data
            sleep(1)



In [8]:
commits = {}

In [37]:
with open('commits_pages.pickle', 'wb') as handle:
    pickle.dump(commit_pages, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i_commits in trange(total_commits, desc='GitHub Commits'):
    try:    
        sha_commit = github_pulls[i_commits].sha
        if sha_commit not in commits:
            commits[sha_commit] = github_commits[i_commits].raw_data
            sleep(1)            
    except:
        print(f"Commit Not Found:{i_commits}")


In [27]:
#commit_test.raw_data['number']bef8d426f279064257416f78d66041c180efa67f
#print("Hola")
commits[list(commits.keys())[0]]

{'sha': '402e551ca27499a9dc211dd6e4eca21d9aa9d089',
 'node_id': 'MDY6Q29tbWl0MjA1ODA0OTg6NDAyZTU1MWNhMjc0OTlhOWRjMjExZGQ2ZTRlY2EyMWQ5YWE5ZDA4OQ==',
 'commit': {'author': {'name': 'Kubernetes Prow Robot',
   'email': 'k8s-ci-robot@users.noreply.github.com',
   'date': '2019-11-12T17:07:01Z'},
  'committer': {'name': 'GitHub',
   'email': 'noreply@github.com',
   'date': '2019-11-12T17:07:01Z'},
  'message': 'Merge pull request #85135 from wojtek-t/delete_unnecessary_conversions_1\n\nEliminate couple unnecessary conversions',
  'tree': {'sha': '5e7b9d4e0d13ba6b8a2996532e08445e64b650d5',
   'url': 'https://api.github.com/repos/kubernetes/kubernetes/git/trees/5e7b9d4e0d13ba6b8a2996532e08445e64b650d5'},
  'url': 'https://api.github.com/repos/kubernetes/kubernetes/git/commits/402e551ca27499a9dc211dd6e4eca21d9aa9d089',
  'comment_count': 0,
  'verification': {'verified': True,
   'reason': 'valid',
   'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJdyua1CRBK7hj4Ov3rIwAAdHIIAJO3X

In [34]:
%%time
commit_pages[30][0].sha in commits

CPU times: user 141 µs, sys: 35 µs, total: 176 µs
Wall time: 470 µs


True